In [1]:
%reload_ext autoreload
%autoreload 2

from sarpu.data_processing import *
from sarpu.paths_and_names import *

import numpy as np
import pandas as pd
import requests

import sklearn.model_selection

# Input Data

In [2]:
# Names and locations
data_folder= "../../Data/"
data_name = "covtype"
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz"

In [3]:
# Creation information
nb_splits = 5
test_size = 0.2

# Prepare folders

In [4]:
# Prepare folders
data_folder_original = original_data_path(data_folder,data_name)
!mkdir -p $data_folder_original
data_folder_processed = processed_data_path(data_folder,data_name)
!mkdir -p $data_folder_processed
data_folder_partitions = partitions_data_path(data_folder,data_name)
!mkdir -p $data_folder_partitions

# Download

In [5]:
#download and unzip data
archive = os.path.join(data_folder_original,url.split("/")[-1])
unprocessed_data_path = ".".join(archive.split(".")[:-1])

if not(os.path.exists(unprocessed_data_path)):
    r = requests.get(url, allow_redirects=True)
    open(archive, 'wb').write(r.content)
    !gzip -f -d $archive

In [6]:
#read data to pandas dataframe


# 6.	Number of Attributes:	12 measures, but 54 columns of data
# 				(10 quantitative variables, 4 binary
# 				wilderness areas and 40 binary
# 				soil type variables)


# 7.	Attribute information:

# Given is the attribute name, attribute type, the measurement unit and
# a brief description.  The forest cover type is the classification 
# problem.  The order of this listing corresponds to the order of 
# numerals along the rows of the database.

# Name                                     Data Type    Measurement                       Description

# Elevation                               quantitative    meters                       Elevation in meters
# Aspect                                  quantitative    azimuth                      Aspect in degrees azimuth
# Slope                                   quantitative    degrees                      Slope in degrees
# Horizontal_Distance_To_Hydrology        quantitative    meters                       Horz Dist to nearest surface water features
# Vertical_Distance_To_Hydrology          quantitative    meters                       Vert Dist to nearest surface water features
# Horizontal_Distance_To_Roadways         quantitative    meters                       Horz Dist to nearest roadway
# Hillshade_9am                           quantitative    0 to 255 index               Hillshade index at 9am, summer solstice
# Hillshade_Noon                          quantitative    0 to 255 index               Hillshade index at noon, summer soltice
# Hillshade_3pm                           quantitative    0 to 255 index               Hillshade index at 3pm, summer solstice
# Horizontal_Distance_To_Fire_Points      quantitative    meters                       Horz Dist to nearest wildfire ignition points
# Wilderness_Area (4 binary columns)      qualitative     0 (absence) or 1 (presence)  Wilderness area designation
# Soil_Type (40 binary columns)           qualitative     0 (absence) or 1 (presence)  Soil Type designation
# Cover_Type (7 types)                    integer         1 to 7                       Forest Cover Type designation



header = [
    "Elevation",
    "Aspect",
    "Slope",
    "Horizontal_Distance_To_Hydrology",
    "Vertical_Distance_To_Hydrology",
    "Horizontal_Distance_To_Roadways",
    "Hillshade_9am",
    "Hillshade_Noon",
    "Hillshade_3pm",
    "Horizontal_Distance_To_Fire_Points",
         ] + list(
    map(lambda i: "Wilderness_Area_"+str(i), range(4))
    ) + list(
    map(lambda i: "Soil_Type"+str(i), range(40))
    ) + ["Cover_Type"]

multival=[]

df = pd.read_csv(unprocessed_data_path, names=header).dropna()
df



,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5
5,2579,132,6,300,-15,67,230,237,140,6031,...,0,0,0,0,0,0,0,0,0,2
6,2606,45,7,270,5,633,222,225,138,6256,...,0,0,0,0,0,0,0,0,0,5
7,2605,49,4,234,7,573,222,230,144,6228,...,0,0,0,0,0,0,0,0,0,5
8,2617,45,9,240,56,666,223,221,133,6244,...,0,0,0,0,0,0,0,0,0,5
9,2612,59,10,247,11,636,228,219,124,6230,...,0,0,0,0,0,0,0,0,0,5


In [7]:
#class distribution

df["Cover_Type"].value_counts()


2    283301
1    211840
3     35754
7     20510
6     17367
5      9493
4      2747
Name: Cover_Type, dtype: int64

In [8]:
# Make cover type 2 the positive class, 48.75% of the instances are positive (learned from previous cell)

df["class"]=0

df.loc[df['Cover_Type'] == 2,"class"]=1
df.groupby('class').count()
df = df.drop("Cover_Type",axis=1)

In [9]:
#binarize multivalued features

for column in multival:
    values = list(set(df[column]))
    if len(values)>2:
        df = binarize(df, column)
    elif len(values)==2:
        df.loc[df[column]==values[0],column]=-1
        df.loc[df[column]==values[1],column]=1
    else: # drop useless features
        print(column, values)
        df=df.drop(column, axis=1)

# df

In [10]:
#normalize
for column in df.columns.values:
    df[column]=pd.to_numeric(df[column])

normalized_df=(df.astype(float)-df.min())/(df.max()-df.min())*2-1
normalized_df["class"] = df["class"]
df = normalized_df
# df

In [11]:
#move class to back

cols = list(df.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('class')) #Remove class from list
df = df[cols+['class']]

df

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,class
0,-0.262631,-0.716667,-0.909091,-0.630637,-0.552972,-0.856681,0.740157,0.826772,0.165354,0.750732,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0
1,-0.268634,-0.688889,-0.939394,-0.696492,-0.568475,-0.890403,0.732283,0.850394,0.188976,0.735675,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0
2,-0.054527,-0.227778,-0.727273,-0.616321,-0.385013,-0.106365,0.842520,0.874016,0.062992,0.706678,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1
3,-0.073537,-0.138889,-0.454545,-0.653543,-0.248062,-0.131657,0.874016,0.874016,-0.039370,0.731772,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1
4,-0.263632,-0.750000,-0.939394,-0.780959,-0.555556,-0.890122,0.732283,0.842520,0.181102,0.720898,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0
5,-0.279640,-0.266667,-0.818182,-0.570508,-0.591731,-0.981172,0.811024,0.866142,0.102362,0.681584,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1
6,-0.252626,-0.750000,-0.787879,-0.613457,-0.540052,-0.822116,0.748031,0.771654,0.086614,0.744319,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0
7,-0.253627,-0.727778,-0.878788,-0.664996,-0.534884,-0.838977,0.748031,0.811024,0.133858,0.736512,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0
8,-0.241621,-0.750000,-0.727273,-0.656407,-0.408269,-0.812842,0.755906,0.740157,0.047244,0.740973,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0
9,-0.246623,-0.672222,-0.696970,-0.646385,-0.524548,-0.821273,0.795276,0.724409,-0.023622,0.737070,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0


In [12]:
# make numpy array

import numpy as np

xy = df.values

x = xy[:,:-1]
y = xy[:,-1].astype(int)

x_pos = x[y==1]
x_neg = x[y==0]



In [13]:
#Save data and true classes
np.savetxt(data_path(data_folder, data_name), x)
np.savetxt(classlabels_path(data_folder, data_name), y,fmt='%d')

# Different dataset partitions (train/test and class prior)

In [14]:
sss = sklearn.model_selection.StratifiedShuffleSplit(n_splits=nb_splits, test_size=test_size, random_state=0)
splits = list(sss.split(x,y))

In [15]:
#save partitions. 0 means not in data, 1 means in train partition, 2 means in test partition

for i, (train,test) in enumerate(splits):
    partition = np.zeros_like(y,dtype=int)
    partition[train]=1
    
    partition[test]=2    
    np.savetxt(partition_path(data_folder,data_name, i), partition, fmt='%d')
   